In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jun  9 08:17:57 2023

@author: Joao

This script reads a training CSV, outputs predictions for the data, and applies
the score metric to compute the final score. 

It also computes a few other known metrics, like top-k accuracy and average power loss.


"""

# 'F5' Começa a debuger o codigo 
# 'F10' Analisar a linha sem entrar no codigo 
# 'F11' Analisar linha  e entrar no codigo 
# 'SHIFT-F11' sair do bloco de codigo atual e continuar a execução


import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
from tqdm import tqdm

X_SIZE = 5      # 5 input samples
N_GPS = 2       # 2 GPSs (unit1 and unit2)
N_GPS_COORD = 2 # 2 GPS coords (latitude & longitude)
N_ARR = 4       # 4 arrays
N_BEAMS = 64    # 64 beams per array
IDX_COL1 = 'unique_index' # index col in the training CSV
IDX_COL2 = 'abs_index'    # index col in the CSVs of the V2V dataset 
                          # this indices are the same, just dif names

def norm_2pi(x):
    """
    Normalize angles in radians to the range of -pi to pi.

    Parameters:
    - x (numpy.ndarray or array-like): Input angles in radians.

    Returns:
    - numpy.ndarray: Angles normalized to the range of -pi to pi.

    The function takes an array of angles in radians and normalizes them to the
    range of -pi to pi. It handles values outside this range by applying modular
    arithmetic to bring them within the specified range.

    Examples:
    >>> import numpy as np
    >>> angles = np.array([-3*np.pi, 2*np.pi, np.pi/2])
    >>> norm_2pi(angles)
    array([ 3.14159265, -3.14159265,  1.57079633])

    Note:
    The function uses modular arithmetic to ensure the normalized angles lie
    within the range of -pi to pi. If an angle is less than -pi after
    normalization, a warning message is printed.
    """
    # -pi to pi
    x_normed = np.empty_like(x)
    x_normed[:] = x
    for i in range(len(x)):
        if abs(x[i]) >= np.pi:
            x_normed[i] = x[i] % (2*np.pi)

            if x[i] >= np.pi:
                x_normed[i] -= 2*np.pi

        while x_normed[i] < -np.pi:
            x_normed[i] += 2*np.pi

        while x_normed[i] > np.pi:
            x_normed[i] -= 2*np.pi

        if x_normed[i] < -np.pi:
            print(f'{i}, {x_normed[i]}')

    return x_normed


def compute_ori_from_pos_delta(lat_deltas, lon_deltas):
    """
    Compute orientation in the range [-pi, pi].

    Parameters:
    - lat_deltas (numpy.ndarray or array-like): Differences in latitudes.
    - lon_deltas (numpy.ndarray or array-like): Differences in longitudes.

    Returns:
    - numpy.ndarray: Orientations corresponding to the differences in latitudes and longitudes.

    The function computes orientation based on differences in latitudes and longitudes.
    The orientation is given in radians and falls within the range of -pi to pi.
    The orientation is calculated using arctan(delta_lat / delta_lon).
    If delta_lon is 0, the orientation is determined based on the sign of delta_lat.

    Thresholds:
    If delta_lat is below a certain threshold (thres_lat), it is considered as 0.

    If lat_deltas and lon_deltas are N x 2, the function computes the difference
    between the two columns.
    If lat_deltas and lon_deltas are N x 1, it computes differences in consecutive samples
    (uses two positions at different times to get orientation).


    Examples:
    >>> import numpy as np
    >>> lat_deltas = np.array([0, 1, -1, 0])
    >>> lon_deltas = np.array([1, 0, 0, -1])
    >>> compute_ori_from_pos_delta(lat_deltas, lon_deltas)
    array([ 0.        ,  1.57079633, -1.57079633,  3.14159265])

    """
    n_samples = len(lat_deltas)
    pose = np.zeros(n_samples)

    for i in range(n_samples):

        delta_lat = lat_deltas[i-1]
        delta_lon = lon_deltas[i-1]

        if delta_lon == 0:
            if delta_lat == 0:
                pose[i] = 0
                continue
            elif delta_lat > 0:
                slope = np.pi / 2
            elif delta_lat < 0:
                slope = -np.pi / 2
        else:
            slope = np.arctan(delta_lat / delta_lon)
            if delta_lat == 0:
                slope = np.pi if delta_lon < 0 else 0
            elif delta_lat < 0 and delta_lon < 0:
                slope = -np.pi + slope
            elif delta_lon < 0 and delta_lat > 0:
                slope = np.pi + slope

        pose[i] = slope

    return pose


def estimate_positions(input_positions, delta_input, delta_output):
    """
    Estimate positions based on input positions using linear interpolation.

    Parameters:
    - input_positions (numpy.ndarray): Input positions with shape (n_samples, n_points, 2).
    - delta_input (float): Time difference between consecutive input samples.
    - delta_output (float): Time difference for the desired output position.

    Returns:
    - numpy.ndarray: Estimated output positions with shape (n_samples, 2).

    The function estimates output positions based on input positions using linear interpolation.
    It assumes that input positions are provided at regular intervals with a specified time difference (delta_input).
    The output positions are estimated at a future time (delta_output) using linear interpolation.

    Examples:
    >>> input_positions = np.array([[[1.0, 2.0], [2.0, 3.0], [3.0, 4.0]],
    ...                              [[4.0, 5.0], [5.0, 6.0], [6.0, 7.0]]])
    >>> delta_input = 1.0
    >>> delta_output = 2.0
    >>> estimate_positions(input_positions, delta_input, delta_output)
    array([[ 5.,  6.],
           [10., 11.]])

    Note:
    - The input_positions array should have shape (n_samples, n_points, 2), where n_samples is the number of samples,
      n_points is the number of input points, and the last dimension represents the (latitude, longitude) coordinates.
    - The function uses linear interpolation to estimate the positions at a future time (delta_output).
    """

        # Calculate the number of samples in the input positions array
    n_samples = input_positions.shape[0]

    # Initialize an array to store the estimated output positions
    out_pos = np.zeros((n_samples, 2))

    # Determine the size of each input position array (number of samples)
    x_size = input_positions.shape[1]

    # Define the time points corresponding to each sample in the input positions array
    x = delta_input * np.arange(x_size)

    # Iterate over each sample to estimate the corresponding output position
    for sample_idx in tqdm(range(n_samples), desc='Estimating input positions'):
        # Extract the input positions for the current sample
        input_pos = input_positions[sample_idx]

        # Perform linear interpolation to estimate latitude and longitude at the output time
        f_lat = scipy.interpolate.interp1d(x, input_pos[:, 0], fill_value='extrapolate')
        f_lon = scipy.interpolate.interp1d(x, input_pos[:, 1], fill_value='extrapolate')

        # Calculate the estimated latitude and longitude at the output time
        out_pos[sample_idx, 0] = f_lat(x[-1] + delta_output)
        out_pos[sample_idx, 1] = f_lon(x[-1] + delta_output)

    # Return the array of estimated output positions
    return out_pos



def predict_beam_uniformly_from_aoa(aoa):
    """
    Predict beams uniformly based on the angles of arrival (AOA).

    Parameters:
    - aoa (numpy.ndarray or array-like): Angles of arrival with shape (N, 1),
      where N is the number of datapoints.

    Returns:
    - numpy.ndarray: Ordered list of indices of the closest predictor points,
      representing the predicted beams. Shape (N, K), where K is the total number
      of beams.

    The function computes the distance of each datapoint to each predictor point
    based on the angles of arrival (AOA). It returns an ordered list of indices
    representing the predicted beams for each datapoint.

    Note:
    - The input aoa should have shape (N, 1), where N is the number of datapoints.
    - The function uses a uniformly distributed set of predictor points in the
      range [-pi, pi] to compute the distance.
    - The output is an array with shape (N, K), where K is the total number of beams.
    - The indices in the output array represent the closest predictor points for each datapoint.

    Examples:
    >>> aoa = np.array([[0.1], [1.5], [-2.0]])
    >>> predict_beam_uniformly_from_aoa(aoa)
    array([[12, 13, 11, ...,  6,  7,  5],
           [11, 12, 10, ...,  5,  6,  4],
           [ 5,  6,  4, ..., 15,  0, 14]])

    """
    beam_predictions = np.zeros_like(aoa)

    beam_ori = np.arange(N_BEAMS * N_ARR) / (N_BEAMS * N_ARR - 1) * 2*np.pi - np.pi

    angl_diff_to_each_beam = aoa.reshape((-1, 1)) - beam_ori

    beam_predictions = np.argsort(abs(angl_diff_to_each_beam), axis=1)

    return beam_predictions


def circular_distance(a, b, l=256, sign=False):
    """
    Compute the circular distance between two beam indices.

    Parameters:
    - a (int): First beam index.
    - b (int): Second beam index.
    - l (int, optional): Total number of beam indices in the circle (default is 256).
    - sign (bool, optional): If True, considers a as predicted and b as truth (default is False).

    Returns:
    - int: Circular distance between the two beam indices.

    The function computes the circular distance between two beam indices, a and b,
    in a circular way. It considers all numbers written in a circle with 'l' numbers,
    then computes the shortest distance between any two numbers.

    Examples:
    >>> circular_distance(0, 5, l=256)
    5
    >>> circular_distance(0, 255, l=256)
    1
    >>> circular_distance(0, 250, l=256)
    6
    >>> circular_distance(0, 127, l=256)
    127

    Note:
    - If 'sign' is True, a is considered as predicted, and b as truth.
    - The distance is returned as a positive value unless 'sign' is True.
    """
    while a < 0:
        a = l - abs(a)
    while b < 0:
        b = l - abs(b)
        
    a = a % l if a >= l else a
    b = b % l if b >= l else b
    
    dist = a - b

    if abs(dist) > l/2:
        dist = l - abs(dist)

    return dist if sign else abs(dist)


def compute_acc(all_beams, only_best_beam, top_k=[1, 3, 5]):
    
    """
    Compute top-k accuracy given predicted and ground truth labels.

    Parameters:
    - all_beams (numpy.ndarray): Predicted beams with shape (N_SAMPLES, N_BEAMS),
      representing either ground truth beams sorted by receive power or predicted beams
      sorted by algorithm's confidence.
    - only_best_beam (numpy.ndarray): Ground truth or predicted optimal beam index with shape (N_SAMPLES, 1).
    - top_k (list, optional): List of integers representing the top-k values for accuracy calculation (default is [1, 3, 5]).

    Returns:
    - numpy.ndarray: Top-k accuracy values.

    The function computes top-k accuracy given predicted and ground truth labels.
    It works bidirectionally, allowing it to handle cases where 'all_beams' can represent
    either the ground truth beams sorted by receive power or the predicted beams sorted by
    the algorithm's confidence of being the best.

    Examples:
    >>> all_beams = np.array([[1, 2, 3], [3, 2, 1], [2, 1, 3]])
    >>> only_best_beam = np.array([[1], [2], [3]])
    >>> compute_acc(all_beams, only_best_beam, top_k=[1, 3])
    array([0.6667, 1.    ])

    Note:
    - 'all_beams' is assumed to have shape (N_SAMPLES, N_BEAMS), where N_SAMPLES is the number of samples,
      and N_BEAMS is the number of beams.
    - 'only_best_beam' is assumed to have shape (N_SAMPLES, 1), representing either the ground truth or predicted optimal beam index.
    - 'top_k' is a list of integers specifying the top-k values for accuracy calculation.
    """
    n_top_k = len(top_k)
    total_hits = np.zeros(n_top_k)

    n_test_samples = len(only_best_beam)
    if len(all_beams) != n_test_samples:
        raise Exception(
            'Number of predicted beams does not match number of labels.')

    # For each test sample, count times where true beam is in k top guesses
    for samp_idx in range(len(only_best_beam)):
        for k_idx in range(n_top_k):
            hit = np.any(all_beams[samp_idx, :top_k[k_idx]] == only_best_beam[samp_idx])
            total_hits[k_idx] += 1 if hit else 0

    # Average the number of correct guesses (over the total samples)
    return np.round(total_hits / len(only_best_beam), 4)


def APL(true_best_pwr, est_best_pwr):
    """
    Calculate the Average Power Loss (APL).

    Parameters:
    - true_best_pwr (numpy.ndarray): True best power values.
    - est_best_pwr (numpy.ndarray): Estimated best power values.

    Returns:
    - float: Average Power Loss.

    The function computes the average of the power wasted by using the predicted beam
    instead of the ground truth optimum beam.

    Examples:
    >>> true_best_pwr = np.array([1.0, 2.0, 3.0])
    >>> est_best_pwr = np.array([1.5, 2.5, 3.5])
    >>> APL(true_best_pwr, est_best_pwr)
    0.1761

    Note:
    - 'true_best_pwr' and 'est_best_pwr' should have the same shape.
    - The Average Power Loss is calculated as the average of 10 * log10(est_best_pwr / true_best_pwr).
    """
    
    return np.mean(10 * np.log10(est_best_pwr / true_best_pwr))


In [3]:
scen_idx = 36
csv_train = r'D:\Python\Multi Modal Beam Prediction with Deep Learning\data\raw\scenario36\deepsense_challenge2023_trainset.csv'
csv_dict_path = rf'D:\Python\Multi Modal Beam Prediction with Deep Learning\data\raw\scenario{scen_idx}\scenario{scen_idx}.p'

with open(csv_dict_path, 'rb') as fp:
    csv_dict = pickle.load(fp)

df_train = pd.read_csv(csv_train)
df_train.head()

,scenario,x1_unique_index,x1_unit1_gps1,x1_unit2_gps1,x1_unit1_rgb5,x1_unit1_rgb6,x2_unique_index,x2_unit1_gps1,x2_unit2_gps1,x2_unit1_rgb5,x2_unit1_rgb6,x3_unique_index,x3_unit1_gps1,x3_unit2_gps1,x3_unit1_rgb5,x3_unit1_rgb6,x4_unique_index,x4_unit1_gps1,x4_unit2_gps1,x4_unit1_rgb5,x4_unit1_rgb6,x5_unique_index,x5_unit1_gps1,x5_unit2_gps1,x5_unit1_rgb5,x5_unit1_rgb6,y1_unique_index,y1_unit1_overall-beam,y1_unit1_pwr1,y1_unit1_pwr2,y1_unit1_pwr3,y1_unit1_pwr4
0,36,2674,scenario36/unit1/gps1/gps_8869_11-46-31.233334...,scenario36/unit2/gps1/gps_33301_11-46-31.25000...,scenario36/unit1/rgb5/frame_11-46-31.205818.jpg,scenario36/unit1/rgb6/frame_11-46-31.205818.jpg,2676,scenario36/unit1/gps1/gps_8871_11-46-31.400000...,scenario36/unit2/gps1/gps_33303_11-46-31.41666...,scenario36/unit1/rgb5/frame_11-46-31.406020.jpg,scenario36/unit1/rgb6/frame_11-46-31.406020.jpg,2678,scenario36/unit1/gps1/gps_8873_11-46-31.600000...,scenario36/unit2/gps1/gps_33305_11-46-31.58333...,scenario36/unit1/rgb5/frame_11-46-31.606222.jpg,scenario36/unit1/rgb6/frame_11-46-31.606222.jpg,2680,scenario36/unit1/gps1/gps_8875_11-46-31.800000...,scenario36/unit2/gps1/gps_33308_11-46-31.83333...,scenario36/unit1/rgb5/frame_11-46-31.806424.jpg,scenario36/unit1/rgb6/frame_11-46-31.806424.jpg,2682,scenario36/unit1/gps1/gps_8877_11-46-32.000000...,scenario36/unit2/gps1/gps_33310_11-46-32.00000...,scenario36/unit1/rgb5/frame_11-46-32.006626.jpg,scenario36/unit1/rgb6/frame_11-46-32.006626.jpg,2687,161,scenario36/unit1/pwr1/pwr_11-46-31.859441.txt,scenario36/unit1/pwr2/pwr_11-46-31.859441.txt,scenario36/unit1/pwr3/pwr_11-46-31.859441.txt,scenario36/unit1/pwr4/pwr_11-46-31.859441.txt
1,36,2675,scenario36/unit1/gps1/gps_8870_11-46-31.300000...,scenario36/unit2/gps1/gps_33302_11-46-31.33333...,scenario36/unit1/rgb5/frame_11-46-31.305919.jpg,scenario36/unit1/rgb6/frame_11-46-31.305919.jpg,2677,scenario36/unit1/gps1/gps_8872_11-46-31.500000...,scenario36/unit2/gps1/gps_33304_11-46-31.50000...,scenario36/unit1/rgb5/frame_11-46-31.506121.jpg,scenario36/unit1/rgb6/frame_11-46-31.506121.jpg,2679,scenario36/unit1/gps1/gps_8874_11-46-31.700000...,scenario36/unit2/gps1/gps_33307_11-46-31.75000...,scenario36/unit1/rgb5/frame_11-46-31.706323.jpg,scenario36/unit1/rgb6/frame_11-46-31.706323.jpg,2681,scenario36/unit1/gps1/gps_8876_11-46-31.900000...,scenario36/unit2/gps1/gps_33309_11-46-31.91666...,scenario36/unit1/rgb5/frame_11-46-31.906525.jpg,scenario36/unit1/rgb6/frame_11-46-31.906525.jpg,2683,scenario36/unit1/gps1/gps_8879_11-46-32.133334...,scenario36/unit2/gps1/gps_33311_11-46-32.08333...,scenario36/unit1/rgb5/frame_11-46-32.106727.jpg,scenario36/unit1/rgb6/frame_11-46-32.106727.jpg,2688,161,scenario36/unit1/pwr1/pwr_11-46-31.959645.txt,scenario36/unit1/pwr2/pwr_11-46-31.959645.txt,scenario36/unit1/pwr3/pwr_11-46-31.959645.txt,scenario36/unit1/pwr4/pwr_11-46-31.959645.txt
2,36,2676,scenario36/unit1/gps1/gps_8871_11-46-31.400000...,scenario36/unit2/gps1/gps_33303_11-46-31.41666...,scenario36/unit1/rgb5/frame_11-46-31.406020.jpg,scenario36/unit1/rgb6/frame_11-46-31.406020.jpg,2678,scenario36/unit1/gps1/gps_8873_11-46-31.600000...,scenario36/unit2/gps1/gps_33305_11-46-31.58333...,scenario36/unit1/rgb5/frame_11-46-31.606222.jpg,scenario36/unit1/rgb6/frame_11-46-31.606222.jpg,2680,scenario36/unit1/gps1/gps_8875_11-46-31.800000...,scenario36/unit2/gps1/gps_33308_11-46-31.83333...,scenario36/unit1/rgb5/frame_11-46-31.806424.jpg,scenario36/unit1/rgb6/frame_11-46-31.806424.jpg,2682,scenario36/unit1/gps1/gps_8877_11-46-32.000000...,scenario36/unit2/gps1/gps_33310_11-46-32.00000...,scenario36/unit1/rgb5/frame_11-46-32.006626.jpg,scenario36/unit1/rgb6/frame_11-46-32.006626.jpg,2684,scenario36/unit1/gps1/gps_8880_11-46-32.200000...,scenario36/unit2/gps1/gps_33313_11-46-32.25000...,scenario36/unit1/rgb5/frame_11-46-32.206828.jpg,scenario36/unit1/rgb6/frame_11-46-32.206828.jpg,2689,161,scenario36/unit1/pwr1/pwr_11-46-32.059402.txt,scenario36/unit1/pwr2/pwr_11-46-32.059402.txt,scenario36/unit1/pw

In [2]:
pd.set_option('display.max_columns', None)

df_train.head()

NameError: name 'df_train' is not defined

In [7]:

# Load all positions
samples_of_scen = np.where(df_train['scenario'] == scen_idx)[0]
n_samples = len(samples_of_scen)

X_SIZE = 5      # 5 input samples
N_GPS = 2       # 2 GPSs (unit1 and unit2)
N_GPS_COORD = 2 # 2 GPS coords (latitude & longitude)
N_ARR = 4       # 4 arrays
N_BEAMS = 64    # 64 beams per array
IDX_COL1 = 'unique_index' # index col in the training CSV
IDX_COL2 = 'abs_index'    # index col in the CSVs of the V2V dataset 
                          # this indices are the same, just dif names
loaded_positions = set()
train_positions = np.zeros((n_samples, X_SIZE, N_GPS, N_GPS_COORD))
y_pos1 = np.zeros((n_samples, N_GPS_COORD))
y_pos2 = np.zeros((n_samples, N_GPS_COORD))
y_pwrs = np.zeros((n_samples, N_ARR, N_BEAMS))

# Loop over each sample in the training data
for sample_idx in tqdm(range(n_samples), desc='Loading data'):
    # Get the current training sample
    train_sample = samples_of_scen[sample_idx]
    
    # Loop over each position index on the X axis
    for x_idx in range(X_SIZE):
        # Find the absolute relative index in the CSV DataFrame for the current training sample
        abs_idx_relative_index = (csv_dict[IDX_COL2] == df_train[f'x{x_idx+1}_'+IDX_COL1][train_sample])
        
        # Fill the training positions for the current sample and current X index
        # GPS positions of 'unit1' and 'unit2' are stored in train_positions
        # Index [0, :] is used for 'unit1' GPS coordinates, and index [1, :] is used for 'unit2' GPS coordinates
        train_positions[sample_idx, x_idx, 0, :] = csv_dict['unit1_gps1'][abs_idx_relative_index]
        train_positions[sample_idx, x_idx, 1, :] = csv_dict['unit2_gps1'][abs_idx_relative_index]

    # Find the index for the ground truth position 'y' in the CSV dictionary DataFrame
    y_idx = (csv_dict[IDX_COL2] == df_train['y1_'+IDX_COL1][train_sample])

    # Store the GPS positions for 'unit1' and 'unit2' corresponding to the ground truth 'y' position
    y_pos1[sample_idx] = csv_dict['unit1_gps1'][y_idx]
    y_pos2[sample_idx] = csv_dict['unit2_gps1'][y_idx]

    # Loop over each antenna array to store the power readings corresponding to the ground truth 'y' position
    for arr_idx in range(N_ARR):
        y_pwrs[sample_idx, arr_idx] = csv_dict[f'unit1_pwr{arr_idx+1}'][y_idx]


Loading data:   0%|          | 0/23187 [00:00<?, ?it/s]

Loading data: 100%|██████████| 23187/23187 [00:28<00:00, 819.55it/s]


In [34]:
# Extract ground truth beams for 'unit1' from the DataFrame, based on the selected samples
y_true_beams = df_train['y1_unit1_overall-beam'].values[samples_of_scen] 

y_pwrs_reshaped = y_pwrs.reshape((n_samples, -1)) # (23187, 256)
# Sort the power readings in descending order to get the indices of the beams with highest power
all_true_beams = np.flip(np.argsort(y_pwrs_reshaped, axis=1), axis=1) # (23187, 256)


In [36]:
y_pwrs_reshaped

array([[0.00012933, 0.00017002, 0.00016716, ..., 0.00015581, 0.0001499 ,
        0.00014879],
       [0.00012251, 0.00016665, 0.00017345, ..., 0.0001511 , 0.00015527,
        0.00015383],
       [0.00012754, 0.00015688, 0.00016276, ..., 0.00015055, 0.00014433,
        0.00015737],
       ...,
       [0.00012266, 0.00014684, 0.00014105, ..., 0.00014967, 0.00015166,
        0.00015794],
       [0.00011438, 0.00014286, 0.00015644, ..., 0.00015184, 0.0001382 ,
        0.0001454 ],
       [0.00012457, 0.0001452 , 0.00015113, ..., 0.00015858, 0.00015953,
        0.00017267]])

In [40]:
# Define the time difference between consecutive input samples and the time difference from the last input to the output
delta_input = 0.2  # time difference between input samples [s]
delta_output = 0.5  # time difference from last input to output [s]

# Estimate positions for 'unit1' GPS data using the given time differences
gps1_est_pos = estimate_positions(train_positions[:, :, 0, :], delta_input, delta_output) # (23187, 5, 2)

# Estimate positions for 'unit2' GPS data using the given time differences
gps2_est_pos = estimate_positions(train_positions[:, :, 1, :], delta_input, delta_output) # (23187, 5, 2)

Estimating input positions: 100%|██████████| 23187/23187 [00:02<00:00, 8067.50it/s]


In [39]:
gps1_est_pos

array([[  33.42170575, -111.93017142],
       [  33.42170569, -111.9301714 ],
       [  33.42170566, -111.93017142],
       ...,
       [  33.37800926, -111.96744191],
       [  33.37801588, -111.96744182],
       [  33.37801599, -111.96744184]])

In [45]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

# Convertendo os dados de entrada para tensores PyTorch
input_positions_tensor = torch.tensor(train_positions[:, :, 0, :], dtype=torch.float32)

class CNNPositionEstimator(nn.Module):
    def __init__(self):
        super(CNNPositionEstimator, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=2, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = torch.max_pool1d(x, kernel_size=x.size(-1))  # Global max pooling
        x = torch.relu(self.fc1(x.squeeze()))
        x = self.fc2(x)
        return x

# Instanciando o modelo
model = CNNPositionEstimator()

# Definindo a função de perda e o otimizador
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Treinamento do modelo
epochs = 10
batch_size = 32

for epoch in range(epochs):
    running_loss = 0.0
    for i in range(0, len(input_positions_tensor), batch_size):
        inputs = input_positions_tensor[i:i+batch_size]
        
        # Zerando os gradientes
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs.permute(0, 2, 1))  # Trocando as dimensões para se adequar ao modelo

        # Calculando a perda
        loss = criterion(outputs, inputs[:, -1, :])  # Comparando a saída com a última posição conhecida

        # Backward pass e otimização
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(input_positions_tensor)}')


Epoch [1/10], Loss: 2.782557746704443
Epoch [2/10], Loss: 5.926279321512369e-07
Epoch [3/10], Loss: 6.149082703315841e-07
Epoch [4/10], Loss: 6.240463482262928e-07
Epoch [5/10], Loss: 6.549802537487114e-07
Epoch [6/10], Loss: 7.301349869152788e-07
Epoch [7/10], Loss: 7.316948440451662e-05
Epoch [8/10], Loss: 0.0013482792166231393
Epoch [9/10], Loss: 0.0013670241568871228
Epoch [10/10], Loss: 0.0014382334439011287


: 